# Structured output

In [27]:
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

True

In [8]:
from pydantic import BaseModel, Field

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)
# Bind responseformatter schema as a tool to the model
model_with_tools = model.bind_tools([ResponseFormatter])
# Invoke the model
ai_msg = model_with_tools.invoke("What is the powerhouse of the cell?")
print(ai_msg)

content='' additional_kwargs={'tool_calls': [{'id': 'call_9OoWjEwP2xwSGCs3VBk2DYRc', 'function': {'arguments': '{"answer":"The powerhouse of the cell is the mitochondrion. Mitochondria are organelles found in the cells of most eukaryotic organisms, and they are responsible for producing energy in the form of adenosine triphosphate (ATP) through the process of cellular respiration.","followup_question":"What is the function of ATP in the cell?"}', 'name': 'ResponseFormatter'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 80, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d8864f8b6b', 'id': 'chatcmpl-BTbyjoq5LCV10CXXIs8xkkGdtaktC', 'service_tier': 'default', 'finish_reason': 'tool_

# Tool Calling

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [5]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [25]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_mh9NfTBHssu5vImDNpSxDAyW', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_3gyg1ZnAyl3wZA3RNOd489kK', 'type': 'tool_call'}]


In [26]:
from langchain_core.messages import ToolMessage

for tool_call in ai_msg.tool_calls:
    tool_name = tool_call["name"].lower()
    args = tool_call["args"]
    tool_result = {"add": add, "multiply": multiply}[tool_name].invoke(args)

    tool_msg = ToolMessage(tool_call_id=tool_call["id"], content=str(tool_result))

    messages.append(tool_msg)

messages
llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 153, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BTdRTGDI5gAW0FY61XNTDYcmwCl31', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--52adb8dd-f8fb-4d84-af54-97fedd547c16-0', usage_metadata={'input_tokens': 153, 'output_tokens': 32, 'total_tokens': 185, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Embeddings

In [29]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize with an embedding model
vector_store = InMemoryVectorStore(embedding=embeddings)

In [31]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

documents = [document_1, document_2]

# vector_store.add_documents(documents=documents)
vector_store.add_documents(documents=documents, ids=["doc1", "doc2"])

['doc1', 'doc2']